In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Define base directory
base_dir = "/Users/bilalnajar/PycharmProjects/capstone/waterfall"

img_size = 64

# Define selected categories for proof of concept
selected_categories = [
    "fm",
    "bluetooth",
    "digital-audio-broadcasting",
    "z-wave",
    "RS41-Radiosonde"
]

# Load only selected categories
images = []
labels = []

for category in selected_categories:
    category_path = os.path.join(base_dir, category)
    if not os.path.isdir(category_path):
        print(f"Skipping missing category: {category}")
        continue

    for img_name in os.listdir(category_path):
        try:
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img.flatten())
            labels.append(category)
        except Exception as e:
            print(f"Error loading image {img_name} from {category}: {e}")
            
# Proceed with training if any images were loaded
if images:
    le = LabelEncoder()
    y = le.fit_transform(labels)

    X = np.array(images)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, stratify=y, random_state=27
    )

    # Train SVM
    svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
    svm_model.fit(X_train, y_train)

    # Evaluate
    y_pred = svm_model.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=le.classes_)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print("Classification Report:\n", report)
    print("Confusion Matrix:\n", conf_matrix)

else:
    print("Image loading failed or no images found.")

Classification Report:
                             precision    recall  f1-score   support

           RS41-Radiosonde       0.68      0.47      0.56        55
                 bluetooth       0.77      0.64      0.70        42
digital-audio-broadcasting       1.00      1.00      1.00        13
                        fm       0.73      0.91      0.81       148
                    z-wave       0.67      0.49      0.56        41

                  accuracy                           0.74       299
                 macro avg       0.77      0.70      0.73       299
              weighted avg       0.73      0.74      0.72       299

Confusion Matrix:
 [[ 26   3   0  26   0]
 [  7  27   0   8   0]
 [  0   0  13   0   0]
 [  3   1   0 134  10]
 [  2   4   0  15  20]]


| Class                      | Precision | Recall | F1   | Notes                                      |
|---------------------------|-----------|--------|------|--------------------------------------------|
| RS41-Radiosonde           | 0.68      | 0.47   | 0.56 | Misclassified heavily as fm                |
| bluetooth                 | 0.77      | 0.64   | 0.70 | Some overlap with fm, but good overall     |
| digital-audio-broadcasting| 1.00      | 1.00   | 1.00 | Perfect separation — very distinctive      |
| fm                        | 0.73      | 0.91   | 0.81 | Strong performance — well represented      |
| z-wave                    | 0.67      | 0.49   | 0.56 | Often confused with fm                     |